In [1]:
%display latex

Let's set up everyting so we can calculate Wigner functions of eigenstates.

In [2]:
N = 3
F = GF(2^N, 'x')
x = F.gen()

In [3]:
basis = [x^3, x^5, x^6]

In [4]:
def toInt(k):
    return list(F).index(k)

In [5]:
def graph(pts):
    m = zero_matrix(SR, 2^N, 2^N)
    for p in pts:
        m[toInt(p[0]), toInt(p[1])] = 1
    return m

In [6]:
def components(k):
    return vector([(k * b).trace() for b in basis])

In [7]:
chi = lambda k: exp(pi * I * int(k.trace()))

y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

def ray(mu):
    return lambda t: mu * t

def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [8]:
ps = PS(ray, [[1,1,1]] * 8)
ps

[ 1  1  1  1  1  1  1  1]
[ 1 -1 -I  1 -I  I  I -1]
[ 1 -I -1  I -I  I  1 -1]
[ 1  1  I  I  I  1  1  I]
[ 1 -I -I  I -1  1  I -1]
[ 1  I  I  1  1  I  1  I]
[ 1  I  1  1  I  1  I  I]
[ 1 -1 -1  I -1  I  I -I]

In [9]:
def Proj(u, v=None):
    if not v:
        v = u
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(2^N)

def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

def D(a, b):
    return phi(a, b) * Z(a) * X(b)

So an actually valid parametrization is given by:
$$
\alpha(\tau) = \sigma \tau^2 + \tau^4,
\quad
\beta(\tau) = \tau + \sigma^5 \tau^4.
$$

In [10]:
param = lambda t: (x * t^2 + t^4, t + x^5 * t^4)

In [11]:
def eig_proj(k):
    eig = zero_matrix(SR, 2^N, 2^N)
    for tau in F:
        eig += chi(tau * k) * D(*param(tau)) / 2^N
    return eig

In [100]:
def Wigner(a, b):
    op = zero_matrix(SR, 2^N, 2^N)
    for ga in F:
        for de in F:
            op += chi(ga * b - de * a) * D(ga, de)
    return op / 2^N

In [101]:
def WignerMatrix(state, kernels=None):
    m = zero_matrix(SR, 2^N, 2^N)
    for a in F:
        for b in F:
            if kernels:
                kernel = kernels[toInt(a)][toInt(b)]
            else:
                kernel = Wigner(a, b)
            m[toInt(a), toInt(b)] = (kernel * state).trace() / 2^N
    return m

In [102]:
kernels = []
for a in F: 
    row = [Wigner(a, b) for b in F]
    kernels.append(row)

In [106]:
eig_proj(x^2)

[     0      0      0      0      0      0      0      0]
[     0    1/4  1/4*I -1/4*I      0      0   -1/4      0]
[     0 -1/4*I    1/4   -1/4      0      0  1/4*I      0]
[     0  1/4*I   -1/4    1/4      0      0 -1/4*I      0]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[     0   -1/4 -1/4*I  1/4*I      0      0    1/4      0]
[     0      0      0      0      0      0      0      0]

In [107]:
graph([param(k) for k in F])

[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0]

In [108]:
WignerMatrix(eig_proj(x^2), kernels)

[  0   0   0   0   0   0   0   0]
[  0   0 1/8 1/8   0   0   0   0]
[  0   0 1/8 1/8   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0 1/8   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0   0]
[  0 1/8   0   0   0   0 1/8   0]

We show that for this degenerate curve the trace property does not hold:

In [18]:
for tau in F:
    for a in F:
        t1 = chi(param(tau)[1] * a)
        t2 = chi(param(a)[1] * tau)
        if t1 == t2:
            print(tau, '\t|\t', a)

0 	|	 0
0 	|	 x
0 	|	 x^2
0 	|	 x + 1
0 	|	 x^2 + x
0 	|	 x^2 + x + 1
0 	|	 x^2 + 1
0 	|	 1
x 	|	 0
x 	|	 x
x 	|	 x^2 + x + 1
x 	|	 x^2 + 1
x^2 	|	 0
x^2 	|	 x^2
x^2 	|	 x^2 + 1
x^2 	|	 1
x + 1 	|	 0
x + 1 	|	 x + 1
x + 1 	|	 x^2 + x
x + 1 	|	 x^2 + 1
x^2 + x 	|	 0
x^2 + x 	|	 x + 1
x^2 + x 	|	 x^2 + x
x^2 + x 	|	 x^2 + 1
x^2 + x + 1 	|	 0
x^2 + x + 1 	|	 x
x^2 + x + 1 	|	 x^2 + x + 1
x^2 + x + 1 	|	 x^2 + 1
x^2 + 1 	|	 0
x^2 + 1 	|	 x
x^2 + 1 	|	 x^2
x^2 + 1 	|	 x + 1
x^2 + 1 	|	 x^2 + x
x^2 + 1 	|	 x^2 + x + 1
x^2 + 1 	|	 x^2 + 1
x^2 + 1 	|	 1
1 	|	 0
1 	|	 x^2
1 	|	 x^2 + 1
1 	|	 1


In [19]:
for a in F:
    for b in F:
        s = sum([chi(a * param(tau)[1] + b * param(tau)[0]) for tau in F])
        if s != 0:
            print(a, '\t', b)

0 	 0
0 	 x^2 + x + 1
x + 1 	 x^2 + x
x + 1 	 1
x^2 + x 	 0
x^2 + x 	 x^2 + x + 1
x^2 + 1 	 x^2 + x
x^2 + 1 	 1


So the property holds for points on the curve. Of course this is the case becuase the curve is commutative after all.

In [20]:
curve = [param(tau) for tau in F]
for p1 in curve:
    for p2 in curve:
        if (p1[0] * p2[1]).trace() != (p1[1] * p2[0]).trace():
            raise Exception

But what is not clear is how the sum is null for points not on the curve. For regular curves we have the property $tr(\alpha f(\beta) = tr(\beta f(\alpha))$, but this does not hold, as we have seen, for degenerate curves.

According to some math that I'm doing, if the points $(a,b)$ of the curve satisfy:
$$a \sigma^5 + b + b^2 \sigma^2 + a^4 = 0,$$
then we can use a Lidl theorem to prove that we obtain a delta function.

In [21]:
for a, b in curve:
    if a * x^5 + b + b^2 * x^2 + a^4 == 0:
        print(a,b)

0 0
x^2 + 1 x^2 + x
x^2 + 1 1
x^2 + x 0
0 x^2 + x + 1
x + 1 1
x^2 + x x^2 + x + 1
x + 1 x^2 + x


In [22]:
for a in F:
    for b in F:
        if a * x^5 + b + b^2 * x^2 + a^4 == 0:
            print(a,b)

0 0
0 x^2 + x + 1
x + 1 x^2 + x
x + 1 1
x^2 + x 0
x^2 + x x^2 + x + 1
x^2 + 1 x^2 + x
x^2 + 1 1


In [23]:
def alpha(tau):
    return x * tau^2 + tau^4

def beta(tau):
    return tau + x^5 * tau^4

So an actually valid parametrization is given by:
$$
\alpha(\tau) = \sigma \tau^2 + \tau^4,
\quad
\beta(\tau) = \tau + \sigma^5 \tau^4.
$$

For our parametrization we have:
$$
\alpha_0 = 0, \alpha_1 = \sigma, \alpha_2 = 1,
\quad
\beta_0 = 1, \beta_1 = 0, \beta_2 = \sigma^5.
$$

In [24]:
alphas = [F(0), x, F(1)]
betas  = [F(1), F(0), x^5]

In [25]:
a = alpha(tau)
b = beta(tau)
alphas[2] * b + betas[2] * a + alphas[1]^2 * b^2 + betas[1]^2 * a^2 + alphas[0]^4 * b^4 + betas[0]^4 * a^4

0

In [26]:
alphas[2] * betas[0] + betas[2] * alphas[0] + alphas[1]^2 * betas[2]^2 + betas[1]^2 * alphas[2]^2 + alphas[0]^4 * betas[1]^4 + betas[0]^4 * alphas[1]^4

0

In [27]:
alphas[2] * betas[1] + betas[2] * alphas[1] + alphas[1]^2 * betas[0]^2 + betas[1]^2 * alphas[0]^2 + alphas[0]^4 * betas[2]^4 + betas[0]^4 * alphas[2]^4

0

So for our parametrization it definitely works.

In [28]:
# linear combinations of tau^2 and tau^4
# equal to zero.
for c in F:
    for d in F:
        ze = True
        for tau in F:
            if c * tau^2 + d * tau^4 != 0:
                ze = False
                break
        if ze:
            print(c,d)

0 0


---
Klimov is asking if for the admissible points $(a,b)$, that is the points that satisfy 
$$
tr(\sigma^4 b) = 0, \quad tr(\sigma^5 a) = 0,
$$
then the trace property holds for all $\tau \in F$:
$$
tr(\alpha(\tau) b) = tr(\alpha(b) \tau),
\quad
tr(\beta(\tau) a) = tr(\beta(a) \tau).
$$
We recall our parametrization: $\alpha(\tau) = \sigma \tau^2 + \tau^4$ and $\beta(\tau) = \tau + \sigma^5 \tau^4$.

In [29]:
def alpha(tau):
    return x * tau^2 + tau^4

def beta(tau):
    return tau + x^5 * tau^4

In [30]:
# admissible points
for a in F:
    for b in F:
        if (x^4 * b).trace() == 0 and (x^5 * a).trace() == 0:
            print((a,b))

(0, 0)
(0, x^2 + x)
(0, x^2 + x + 1)
(0, 1)
(x + 1, 0)
(x + 1, x^2 + x)
(x + 1, x^2 + x + 1)
(x + 1, 1)
(x^2 + x, 0)
(x^2 + x, x^2 + x)
(x^2 + x, x^2 + x + 1)
(x^2 + x, 1)
(x^2 + 1, 0)
(x^2 + 1, x^2 + x)
(x^2 + 1, x^2 + x + 1)
(x^2 + 1, 1)


In [31]:
a = 0
b = x^2 + x
for tau in F:
    print(tau, '\t', (alpha(tau) * b).trace(), (alpha(b) * tau).trace())

0 	 0 0
x 	 1 0
x^2 	 1 0
x + 1 	 0 0
x^2 + x 	 0 0
x^2 + x + 1 	 1 0
x^2 + 1 	 0 0
1 	 1 0


So the property does not hold for all $\tau$ even for the addmissible points. For example consider the admissible point $(0,x^2+x) = (0,\sigma^4)$ and $\tau = x$, then we have $tr(\alpha(\tau) (x^2+x)) = tr(x+1) = 1$, but $tr(\alpha(x^2+x) \tau) = tr(0) = 0$.

In [32]:
(alpha(x) * (x^2+x)).trace()

1

In [33]:
(alpha(x^2+x) * x).trace()

0

In [34]:
x^4

x^2 + x

In [35]:
(x^4 * x^5).trace()

0

In [36]:
for a in F:
    for b in F:
        if b + x^5 * a + x^2 * b^2 + a^4 == 0:
            print((a,b))

(0, 0)
(0, x^2 + x + 1)
(x + 1, x^2 + x)
(x + 1, 1)
(x^2 + x, 0)
(x^2 + x, x^2 + x + 1)
(x^2 + 1, x^2 + x)
(x^2 + 1, 1)


In [37]:
curve

[(0, 0),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (x^2 + x, 0),
 (0, x^2 + x + 1),
 (x + 1, 1),
 (x^2 + x, x^2 + x + 1),
 (x + 1, x^2 + x)]

In [38]:
for a in F:
    for k in range(2^N):
        if (a^(2^k)).trace() != a.trace():
            raise Exception

In [39]:
for k in F:
    if k.trace() == 0:
        print(k)

0
x
x^2
x^2 + x


In [40]:
for k in F:
    for b in F:
        if k == b - b^2:
            print(k)
            break

0
x
x^2
x^2 + x


In [41]:
curve

[(0, 0),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (x^2 + x, 0),
 (0, x^2 + x + 1),
 (x + 1, 1),
 (x^2 + x, x^2 + x + 1),
 (x + 1, x^2 + x)]

$\alpha(\tau) = \sigma \tau^2 + \tau^4$, so $\alpha_0 = 0, \alpha_1 = \sigma$ and $\alpha_2 = 1$. And for $\beta(\tau) = \tau + \sigma^5 \tau^4$ we have $\beta_0 = 1, \beta_1 = 0$ and $\beta_2 = \sigma^5$.

In [42]:
alphas

[0, x, 1]

In [43]:
betas

[1, 0, x^2 + x + 1]

In [44]:
vector(betas) * x

(x, 0, x^2 + 1)

In [45]:
def Lidl(a,b):
    s = vector(alphas) * b + vector(betas) * a
    return [k^(2^(N-1-m)) for m, k in enumerate(s)]

In [46]:
for a, b in curve:
    print(Lidl(a,b))

[0, 0, 0]
[x + 1, x + 1, 0]
[x + 1, x^2, x^2 + x + 1]
[x^2, 0, x^2]
[0, x^2 + x + 1, x^2 + x + 1]
[x^2 + x + 1, x^2, x + 1]
[x^2, x^2 + x + 1, x + 1]
[x^2 + x + 1, x + 1, x^2]


---

Let $(a,b)$ be an arbitrary point. If $\sum_{m=0}^{n-1} tr(\alpha_m b - \beta_m a) = 0$, is true that $\sum_{m=0}^{n-1} tr\left[(\alpha_m b - \beta_m a) \tau^{p^m}\right] = 0$ for all $\tau \in F$?

In [73]:
points_to_test = []
for a in F:
    for b in F:
        if sum([(alphas[N-1-i] * b - betas[N-1-i] * a)^(2^i) for i in range(N)]) == 0:
            points_to_test.append((a,b))

In [74]:
set(points_to_test) == set(curve)

True

In [75]:
for a, b in points_to_test:
    print(sum([(alphas[i] * b - betas[i] * a).trace() for i in range(N)]) == 0)

True
True
True
True
True
True
True
True


In [86]:
for a, b in points_to_test:
    for tau in F:
        s  = [((alphas[k] * b - betas[k] * a) * tau^(2^k)).trace() for k in range(N)]
        if sum(s) != 0:
            raise Exception

So it computationally verifies that this is true. Therefore since 
$$
0 = \sum_{m=0}^{n-1} tr\left[(\alpha_m b - \beta_m a) \tau^{p^m}\right]
= tr(\alpha(\tau) b - \beta(\tau) a),
$$
holds for all $\tau \in F$, then the point must be in curve.

In [87]:
points_to_test = []
for a in F:
    for b in F:
        if sum([(alphas[i] * b - betas[i] * a).trace() for i in range(N)]) == 0:
            points_to_test.append((a,b))

In [88]:
len(points_to_test)

32

In [90]:
for a, b in points_to_test:
    for tau in F:
        s  = [((alphas[k] * b - betas[k] * a) * tau^(2^k)).trace() for k in range(N)]
        if sum(s) != 0:
            raise Exception(a,b,tau)

Exception: (0, x, x)

In [98]:
x.trace() == (x^8).trace()

True